In [30]:
import pandas as pd
import requests
from common import *

In [31]:
def get_tmFc() -> str: # 06시, 18시 가능
    now = datetime.now()
    if 0 <= now.hour < 6: # 전날 18시를 가져와야 함
        yesterday = (now + timedelta(days=-1))
        return yesterday.strftime('%Y%m%d1800')

    elif 6 <= now.hour < 18: # 당일 06시를 가져와야 함
        return now.strftime('%Y%m%d0600')

    elif 18 <= now.hour <= 24: # 당일 18시를 가져와야 함
        return now.strftime('%Y%m%d1800')

get_tmFc()

'202304221800'

In [32]:
temp_url = 'http://apis.data.go.kr/1360000/MidFcstInfoService/getMidTa'
yooksang_url = 'http://apis.data.go.kr/1360000/MidFcstInfoService/getMidLandFcst'

In [33]:
def get_joongi_api(url: str, regId: str) -> list[dict]:
    params = {'ServiceKey': get_servicekey(),
              'pageNo': '1',
              'numOfRows': '100',
              'dataType': 'JSON',
              'regId' : regId,
              'tmFc': get_tmFc()}

    response = requests.get(url=url, params=params).json()

    if response['response']['header']['resultCode'] != '00':
        ErrorMessage = response['response']['header']['resultMsg']
        raise GetApiError(ErrorMessage)

    return response['response']['body']['items']['item']

In [43]:
# def get_datas(regId: str) -> tuple[list, list]:
#     yooksang_data = get_joongi_api(url=yooksang_url, regId=regId)
#     temp_data = get_joongi_api(url=temp_url, regId=regId)
#     return yooksang_data, temp_data


# get_joongi_api(url=yooksang_url, regId = '11D20602')
# yooksang_data, temp_data = get_datas('11E10103')
get_joongi_api(url=temp_url, regId='1.1E+102') # 독도, 울릉도는 ?????????

GetApiError: NO_DATA

In [44]:
get_joongi_api(url=temp_url, regId='11D20602')

[{'regId': '11D20602',
  'taMin3': 10,
  'taMin3Low': 0,
  'taMin3High': 0,
  'taMax3': 16,
  'taMax3Low': 0,
  'taMax3High': 0,
  'taMin4': 9,
  'taMin4Low': 0,
  'taMin4High': 0,
  'taMax4': 15,
  'taMax4Low': 0,
  'taMax4High': 0,
  'taMin5': 8,
  'taMin5Low': 0,
  'taMin5High': 0,
  'taMax5': 19,
  'taMax5Low': 0,
  'taMax5High': 0,
  'taMin6': 12,
  'taMin6Low': 0,
  'taMin6High': 0,
  'taMax6': 21,
  'taMax6Low': 0,
  'taMax6High': 0,
  'taMin7': 12,
  'taMin7Low': 0,
  'taMin7High': 0,
  'taMax7': 21,
  'taMax7Low': 0,
  'taMax7High': 0,
  'taMin8': 13,
  'taMin8Low': 0,
  'taMin8High': 0,
  'taMax8': 19,
  'taMax8Low': 0,
  'taMax8High': 0,
  'taMin9': 12,
  'taMin9Low': 0,
  'taMin9High': 0,
  'taMax9': 21,
  'taMax9Low': 0,
  'taMax9High': 0,
  'taMin10': 12,
  'taMin10Low': 0,
  'taMin10High': 0,
  'taMax10': 20,
  'taMax10Low': 0,
  'taMax10High': 0}]

In [ ]:
get_joongi_api(url=yooksang_url, regId='')

In [58]:
def split_comma_csv():
    df_yooksang = pd.read_csv('중기육상예보.csv', encoding='cp949')
    df_yooksang = df_yooksang.assign(구역=df_yooksang['구역'].str.split(',')).explode('구역').reset_index(drop=True)
    df_yooksang = df_yooksang.applymap(lambda x: x.strip())
    df_yooksang.to_csv('code_yooksang.csv', encoding='utf-8')

split_comma_csv()

In [ ]:
# 2023-04-22 | 지역 | 하늘 | 강수확률 | 최저기온 | 최고기온 | 관광지수